# 🔹 VersaMind – Smart Document Summarizer & Email Drafter.
Powered by Microsoft Phi-3

“Versa” for versatile, and “Mind” for intelligence – designed to simplify tasks through AI.

# 💼 Problem Statement
In today’s fast-paced world, professionals, students, and organizations deal with information overload—long documents, detailed reports, and endless content creation demands.

Manually reading, summarizing, and writing formal content consumes valuable time, leading to reduced productivity and delays.

# ⚙️ VersaMind – Your AI-Powered Solution
VersaMind leverages the Phi-3-mini-4k-instruct LLM from Microsoft to provide:

📄 Smart Summarization – Instantly summarize PDF, DOCX, and TXT files into concise, easy-to-understand content.

✉️ AI-Powered Content Drafting – Create formal, professional, friendly, or casual emails and written content in seconds.

🌍 How It’s Useful in Today’s Society

✔️ Boosts Productivity – Saves hours of reading and writing, freeing up time for more strategic tasks.

✔️ Accessible for All – Whether you're a student, professional, or content creator, VersaMind simplifies complex content and accelerates drafting.

✔️ Improves Communication – Helps craft clear, well-structured emails or messages tailored to any tone, enhancing business and personal interactions.

✔️ Supports Learning – Summarizes academic or technical material into simpler terms, aiding comprehension and retention.

✔️ Eco-Friendly – Reduces reliance on printed documents by enabling quick digital content consumption and sharing.

# 🧩 Part 1: Setup, Utilities, and Summarization Feature

What it does:

Installs required libraries.

Loads the AI model with 4-bit quantization.

Reads PDF, DOCX, and TXT files.

Chunks long text.

Generates summaries.

Cleans and refines text output.

Note: our Colab T4 GPU has ~14.7 GB VRAM, and the Phi-3 model is large. It consumes most of the memory during loading and inference. To avoid GPU memory errors, we loaded Phi-3 in lightweight 4-bit quantization, which preserves accuracy while reducing memory usage — perfect for Colab environments.

🚀 Setup: Install Required Packages


*   -U: Ensures the packages are updated to the latest versions.

*   bitsandbytes: A library for efficient quantization of deep learning models, reducing memory usage.

*   transformers: A Hugging Face library for working with pre-trained NLP models like GPT, BERT, etc.
*   accelerate: Helps optimize deep learning models for faster training and inference on different hardware (CPU/GPU/TPU).


*   
gradio: A tool to create web-based UIs for AI models, making it easy to interact with them.









In [ ]:
!pip install -U bitsandbytes transformers accelerate gradio
!pip install PyMuPDF python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

📦 Import Libraries










*   torch: PyTorch, a deep learning framework used for training and running AI models.
*   AutoTokenizer: Loads a tokenizer for processing text input (tokenizing text before feeding it into a model).


*   AutoModelForCausalLM: Loads a pre-trained causal language model (used for text generation, like GPT models).
*   
BitsAndBytesConfig: Helps configure quantization (reducing model size to run efficiently on lower-end hardware).


*   gradio: A library for creating simple web UIs to interact with AI models.
*   
fitz (PyMuPDF): A library for working with PDF files (reading, extracting text, and modifying PDFs).

*   
docx (python-docx): A library to read, write, and modify Microsoft Word (.docx) documents.









In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gradio as gr
import fitz  # PyMuPDF for PDF
import docx  # For DOCX files

Device Setup & Model Loading







*   Device Setup: Detects whether a GPU (CUDA) is available; otherwise, defaults to CPU.

*   4-bit Quantization: Reduces model size and memory usage using NF4 quantization and double quantization for efficiency.

*   Model & Tokenizer Loading:



*   Loads Microsoft's Phi-3 Mini 4K Instruct model with 4-bit quantization.
*   Fetches the tokenizer to process text input.


*   
Device Optimization: Automatically assigns the model to GPU (if available) or CPU for best performance.








In [ ]:
# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device set to: {device}")

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load model and tokenizer
model_id = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

Device set to: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

📄 Helper: Chunk Long Text

The function chunk_text() splits a long text into smaller chunks (max ~1000 tokens each) without breaking sentences.

How it Works:


*  
Splits the text into sentences using . as a delimiter.


*   Adds sentences to a chunk until the size limit (max_tokens) is reached.


*   Stores the chunk and starts a new one when the limit is exceeded.


* Ensures the last chunk is added before returning the final list.   







In [ ]:
def chunk_text(text, max_tokens=1000):
    sentences = text.split('. ')
    chunks, current_chunk = [], ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_tokens:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

🤖 Generate AI Response

The function generate_response(prompt) generates AI-driven text responses based on a given prompt.

How it Works:



*   
Tokenizes the input and moves it to the model's device (CPU/GPU).

*   Generates text using the model with controlled randomness (temperature, top-k, top-p).

*   
Prevents repetition of 3-word phrases for fluency.





*   Decodes the output into human-readable text.








In [ ]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            no_repeat_ngram_size=3
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


📚 Read Files (PDF, DOCX, TXT)

This script extracts, cleans, and formats text from PDF/DOCX files by fixing typos, removing duplicates, and ensuring proper sentence structure.

Key Functions:


1.   Extract Text:


*   
Reads PDF using PyMuPDF (fitz).
*   Reads DOCX using python-docx.


2.   Fix Typos:

*   Corrects common name misspellings using regex replacements.



3.   Remove Duplicates:

*   Eliminates repeated sentences to improve text clarity.



4.   
Format Sentences:


*   Ensures proper sentence endings and removes unnecessary newlines.





In [ ]:
import re

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text


# Fix common name typos
def fix_name_typos(text):
    replacements = {
        "Gisbund": "Gisburn",
        "Gissburn": "Gisburn",
        "Gisbrown": "Gisburn",
        "Grindley": "Grindle",
        "Grindly": "Grindle",
        "Garbuck": "Gisburn",
        "Giesurn": "Gisburn",
        "Gaiesurn": "Gisburn",
        "Rickmam": "Rickham",
        "Rickmham": "Rickham",
        "Strud": "Stroud",
        "Mrs. Studrd": "Mrs. Stroud",
        "Mrs. Pardiggler": "Mrs. Stroud",
        "Mr. Strud": "Mr. Stroud",
        "Mrs. Pardiggle": "Mrs. Stroud"
    }

    for wrong, correct in replacements.items():
        text = re.sub(rf"\b{wrong}\b", correct, text)
    return text

# Remove repeated sentences
def remove_repeated_sentences(text):
    sentences = text.split('. ')
    seen = set()
    cleaned = []
    for sentence in sentences:
        sentence_clean = sentence.strip()
        if sentence_clean and sentence_clean not in seen:
            cleaned.append(sentence_clean)
            seen.add(sentence_clean)
    return '. '.join(cleaned).strip()

# Ensure clean sentence endings
def clean_endings(text):
    if not text.endswith('.'):
        text += '.'
    return text.replace('\n', ' ').strip()


 Summarization Function

 This script reads, summarizes, and cleans text from PDF, DOCX, and TXT files using an AI model.

🔹 Key Steps:

1️⃣ Extracts text from the uploaded file.

2️⃣ Splits large text into smaller chunks (max 1000 tokens).

3️⃣ Generates AI-based summaries for each chunk.

4️⃣ Post-processes the summary by fixing typos, removing duplicates, and ensuring proper formatting.

5️⃣ Handles errors gracefully using traceback.

In [ ]:
import traceback

def summarize_file(file):
    try:
        if file.name.endswith(".pdf"):
            text = read_pdf(file.name)
        elif file.name.endswith(".docx"):
            text = read_docx(file.name)
        elif file.name.endswith(".txt"):
            with open(file.name, "r", encoding="utf-8") as f:
                text = f.read()
        else:
            return "Unsupported file format. Upload .txt, .pdf, or .docx files only."

        chunks = chunk_text(text, max_tokens=1000)
        combined_summary = ""
        for chunk in chunks:
            prompt = f"Summarize the following text in simple terms:\n\n{chunk}\n\nSummary:"
            summary = generate_response(prompt)
            cleaned_summary = summary.split("Summary:")[-1].strip()
            combined_summary += f"{cleaned_summary} "

        # 🔧 Post-Processing Pipeline
        combined_summary = fix_name_typos(combined_summary)
        combined_summary = remove_repeated_sentences(combined_summary)
        combined_summary = clean_endings(combined_summary)

        return combined_summary

    except Exception as e:
        return f"Error occurred:\n{traceback.format_exc()}"


# ✍️ Part 2: Email Drafting Feature

💬 Draft Content Function

This function automates professional email writing based on a given topic and tone while refining the AI-generated output.

🔹 Key Steps:

1️⃣ Generates an AI-based email draft using a structured prompt.

2️⃣ Cleans and formats the response by:


*   Fixing typos (e.g., "responsibilled" → "responsibilities").

*   
Removing unwanted AI instructions (e.g., "Task:", "Write a").
*   
Ensuring the subject line is correctly formatted.


*   
Deleting generic placeholders (e.g., [Company Name]).

*   
Checking for proper sentence endings and adding a signature if missing.












3️⃣ Handles errors gracefully using traceback.

In [ ]:
##This function can give response to any prompt asked in email draft and create email for that query.
def draft_content(topic, tone):
    try:
        prompt = (
            f"Write a {tone.lower()} email about the following topic:\n\n{topic}\n\n"
            f"Only output the email content, including a professional closing and signature line."
        )

        response = generate_response(prompt).strip()

        # 🔧 Remove leaked prompt before "Subject:"
        if "Subject:" in response:
            response = response.split("Subject:", 1)[-1].strip()
            response = "Subject: " + response

        # 🔧 Auto-correct known typos
        response = response.replace("responsibilled", "responsibilities")
        response = response.replace("Prime Minster", "Prime Minister")
        response = response.replace("Moddi", "Modi")
        response = response.replace("India'", "India's")

        # 🔧 Stop if AI starts another task or appends instructions
        stop_phrases = [
            "Write a", "Instruction:", "Task:", "Next:", "Question:"
        ]
        for phrase in stop_phrases:
            if phrase in response:
                response = response.split(phrase)[0].strip()

        # 🔧 Remove unwanted signature fields
        for unwanted in ["[Title]", "[Company Name]"]:
            response = response.replace(unwanted, "").strip()

        # 🔧 Ensure clean sentence or proper email signature ending
        signature_phrases = ["Sincerely,", "Regards,", "Best regards,", "Thank you,", "Yours sincerely,", "Warm regards,"]
        has_signature = any(sig in response for sig in signature_phrases)

        if not has_signature:
            if not response.endswith(('.', '!', '?')):
                last_period = response.rfind('.')
                if last_period != -1:
                    response = response[:last_period+1]
                else:
                    response += "."

        else:
            # Remove trailing text after signature if it leaks
            lines = response.splitlines()
            for i, line in enumerate(lines):
                if any(sig in line for sig in signature_phrases):
                    response = "\n".join(lines[:i+2])  # Keep signature + name
                    break

        # 🔧 Remove leaked prompts or instructions AFTER the signature
        leaked_phrases = [
            "Compose an in-depth", "Please write", "Generate a",
            "Write an analysis", "Create a report", "Answer the following"
        ]
        for phrase in leaked_phrases:
            if phrase in response:
                response = response.split(phrase)[0].strip()

        return response

    except Exception as e:
        import traceback
        return f"Error occurred:\n{traceback.format_exc()}"


In [ ]:
##Where this function gives a sophisticated output when a general question is asked.
import re

def draft_content(topic, tone):
    try:
        # 🚫 Detect general questions or unrelated prompts
        question_keywords = ["what", "why", "how", "when", "who", "which", "is", "are", "do", "does", "should"]
        if topic.strip().endswith("?") or re.match(r"^\s*(" + "|".join(question_keywords) + r")\b", topic.strip().lower()):
            return (
                "📝 This email drafting tool is designed for creating professional or friendly emails based on a specific topic.\n"
                "It looks like you've entered a general question. Please use a different tool or interface for general queries."
            )

        prompt = (
            f"Write a {tone.lower()} email about the following topic:\n\n{topic}\n\n"
            f"Only output the email content, including a professional closing and signature line."
        )

        response = generate_response(prompt).strip()

        # 🔧 Remove leaked prompt before "Subject:"
        if "Subject:" in response:
            response = response.split("Subject:", 1)[-1].strip()
            response = "Subject: " + response

        # 🔧 Auto-correct known typos
        response = response.replace("responsibilled", "responsibilities")

        # 🔧 Stop if AI starts another task
        for stop_phrase in ["Write a", "Instruction:", "Task:", "Next:", "Question:"]:
            if stop_phrase in response:
                response = response.split(stop_phrase)[0].strip()

        # 🔧 Remove unwanted signature fields
        for unwanted in ["[Title]", "[Company Name]"]:
            response = response.replace(unwanted, "").strip()

        # 🔧 Ensure clean sentence ending (if no signature exists)
        signature_phrases = ["Sincerely,", "Regards,", "Best regards,", "Thank you,", "Yours sincerely,"]
        has_signature = any(sig in response for sig in signature_phrases)

        if not has_signature:
            if not response.endswith(('.', '!', '?')):
                last_period = response.rfind('.')
                if last_period != -1:
                    response = response[:last_period+1]
                else:
                    response += "..."

        return response

    except Exception as e:
        import traceback
        return f"Error occurred:\n{traceback.format_exc()}"


🔹 AI Response Generation Function

This function creates structured AI-generated text with controlled length and coherence.

Key Features:

✅ Processes input using tokenization.

✅ Generates text with constraints to prevent randomness and repetition.

✅ Ensures clean stopping using an end-of-sequence token.

✅ Decodes output into readable text.

In [ ]:

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,  # Enough for full email + signature
            temperature=0.5,
            top_p=0.8,
            do_sample=False,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
            eos_token_id=tokenizer.eos_token_id  # Stops cleanly
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


🌐 Unified Gradio Interface – VersaMind

In [ ]:
# --- GRADIO INTERFACE ---
with gr.Blocks(title="🧠 VersaMind – Smart Summarizer & Email Drafter") as app:
    gr.Markdown("# 🧠 VersaMind – Smart Summarizer & Email Drafter")
    gr.Markdown("Upload documents or enter a topic to quickly get AI-generated summaries or professional emails!")

    with gr.Tab("📄 Document Summarizer"):
        file_input = gr.File(label="📄 Upload .txt, .pdf, or .docx file")
        summary_output = gr.Textbox(label="📝 Summary", lines=10)
        summarize_btn = gr.Button("Summarize")
        summarize_btn.click(fn=summarize_file, inputs=file_input, outputs=summary_output)

    with gr.Tab("✉️ Email Drafter"):
        topic_input = gr.Textbox(label="📝 Enter Topic or Prompt", lines=2)
        tone_input = gr.Radio(["Formal", "Friendly", "Professional", "Casual"], label="Select Style/Tone")
        draft_output = gr.Textbox(label="🖋️ Drafted Email", lines=10)
        draft_btn = gr.Button("Draft Email")
        draft_btn.click(fn=draft_content, inputs=[topic_input, tone_input], outputs=draft_output)

app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://74be79d8965a0a6dde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
